In [33]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [ ]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, AgentTool, FunctionTool
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.adk.agents import ParallelAgent, SequentialAgent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [35]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [36]:
def make_booking(place: str, date: str, time: str) -> str:
    """Make a booking at a specified place, date, and time."""
    return f"✅ Booking confirmed at {place} on {date} at {time}"

booking_tool = FunctionTool(make_booking)

In [42]:
def get_location_context(location: str = None) -> str:
    """Get contextual information about a location (simulated MCP tool)."""
    if not location:
        return "Please specify a location to get information about."
    
    # This would connect to a real MCP server in production
    location_info = {
        "weather": "Typically varies by season",
        "timezone": "Local time",
        "currency": "Local currency", 
        "language": "Local language",
        "emergency": "Local emergency numbers"
    }
    
    return f"📍 Location context for {location}: {location_info}"

location_tool = FunctionTool(get_location_context)

In [43]:
booking_agent = Agent(
    name="booking_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a booking concierge agent. Your role is to:
- Help users make reservations and bookings
- Ask for missing information (place, date, time)
- Confirm booking details before proceeding
- Use the make_booking tool when all information is available
- Be friendly and professional""",
    tools = [make_booking]
)
info_agent = Agent(
    name="info_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are an information concierge agent. Your role is to:
- Provide accurate, up-to-date information about places, services, and general knowledge
- Use Google Search to find current information when needed
- Present information in a clear, organized manner
- Cite sources when using searched information""",
    tools = [google_search]
)

recommendation_agent = Agent(
    name="recommendation_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a recommendation concierge agent. Your role is to:
- Recommend activities, places, restaurants, hotels, etc.
- Provide personalized suggestions based on user preferences
- Consider factors like budget, location, and interests
- Use Google Search to find current recommendations
- Provide multiple options when possible""",
    tools = [google_search]
)


In [44]:
router_agent = Agent(
    name="router",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are the concierge router. Your responsibilities:

1. GREETING: If the user is starting a new conversation, give a warm welcome:
   "Hello! I'm Martin, your personal concierge. How can I assist you today? I can help with bookings, information, or recommendations!"

2. ROUTING: Analyze the user's request and categorize it as:
   - "booking": For making reservations, appointments, bookings
   - "information": For factual questions, details about places/services
   - "recommendation": For suggestions on what to do, where to go, etc.

3. CONTEXT: Maintain conversation context and be conversational.

Examples:
- "I want to book a table" → "booking"
- "Tell me about Paris" → "information" 
- "What should I do in Tokyo?" → "recommendation"
- "Hi" → greeting + ask how to help

Always respond naturally while categorizing the request.""",
    output_key="task_type"
)
distributor_agent = Agent(
    name="distributor",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Based on the task type from the router, handle the request appropriately: {task_type}

If it's a greeting or general conversation, respond warmly and helpfully.
If it requires specialized assistance, route to the appropriate agent.

Remember: You are Martin the Concierge - be professional, friendly, and maintain conversation flow.""",
    tools=[AgentTool(recommendation_agent),AgentTool(booking_agent),AgentTool(info_agent), google_search] 
)

In [40]:
workers = ParallelAgent(
    name="workerGroup",
    sub_agents=[booking_agent, info_agent, recommendation_agent]
)


In [45]:
concierge_agent = SequentialAgent(
    name="martin_concierge",
    sub_agents=[router_agent, distributor_agent],
    instruction="""You are Martin, a professional concierge assistant. You provide:
- Booking services (restaurants, hotels, appointments)
- Information services (facts, details, current info) 
- Recommendation services (suggestions, ideas, plans)
- Location-based assistance

Always be polite, professional, and maintain conversation context. Remember user preferences and previous interactions.""",
    tools=[booking_tool, google_search, location_tool]
)


In [46]:
APP_NAME = "martin_concierge"
USER_ID = "user_001"

session_service = InMemorySessionService()

runner = Runner(
    agent=concierge_agent, 
    app_name=APP_NAME, 
    session_service=session_service
)

print("✅ Martin Concierge Agent initialized!")
print("   - Name: Martin Concierge")
print("   - Capabilities: Booking, Information, Recommendations")
print("   - Features: Context memory, Location awareness, Web search")

✅ Martin Concierge Agent initialized!
   - Name: Martin Concierge
   - Capabilities: Booking, Information, Recommendations
   - Features: Context memory, Location awareness, Web search


In [49]:
async def run_concierge_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "concierge_chat",
):
    print(f"\n{'='*60}")
    print(f"🤵 Martin Concierge - Session: {session_name}")
    print(f"{'='*60}")
    
    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
        print("🌟 New session started!")
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
        print("📖 Continuing existing session...")

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\n👤 You: {query}")

            # Convert the query string to the ADK Content format
            query_content = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query_content
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"🤵 Martin: {event.content.parts[0].text}")
    else:
        print("No queries provided!")
    
    print(f"\n{'='*60}")

In [51]:
# Test realistic concierge scenarios
print("Testing Martin Concierge with realistic scenarios...")

await run_concierge_session(
    runner,
    [
        "Hi Martin, I need help with bookings",
        "I want to book a hotel in Paris for 3 nights next week",
        "Also need a restaurant reservation for a romantic dinner",
        "What are some good areas to stay in Paris?",
        "What are the best sites to visit nearby?"
    ],
    "paris_trip_help",
)

Testing Martin Concierge with realistic scenarios...

🤵 Martin Concierge - Session: paris_trip_help
🌟 New session started!

👤 You: Hi Martin, I need help with bookings
🤵 Martin: Okay, I can help you with bookings. What would you like to book?
🤵 Martin: Hi Martin, I need help with bookings.
call booking_agent
🤵 Martin: Hello! I'm ready to help you with your bookings. What can I assist you with today?
🤵 Martin: Hello! I'm happy to help you with your bookings. What would you like to book?
🤵 Martin: I can help you with bookings. What would you like to book?

👤 You: I want to book a hotel in Paris for 3 nights next week
🤵 Martin: call booking_agent
🤵 Martin: I can help you with that. What dates would you like to book your stay for?
🤵 Martin: Great! To help me book your hotel in Paris, could you please provide me with the exact dates for your 3-night stay next week?
🤵 Martin: Great! Paris is a wonderful choice. To help me find the best hotel for you, could you please tell me your preferred c